In [1]:
import pandas as pd
import numpy as np

import datetime as dt
import seaborn as sns
import re

In [2]:
from statsmodels.graphics.mosaicplot import mosaic
import matplotlib.pyplot as plt
%matplotlib inline

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams["font.family"]

['NanumGothic']

In [3]:
data = pd.read_excel('./raw data/train_team.xlsx')
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035


# 역간소요시간 구하기

- 이전역부터 현재역까지의 걸린 시간 구하기: 현역의 계획도전일시 - 이전역의 계획출발일시
- ex) 20191101 금 K1 구일역의 행에 대해서는 구로부터 구일역까지 걸리는 시간을 매칭!

In [4]:
temp = pd.DataFrame()
temp['계획도착일시'] = data['계획도착일시']
temp['계획출발일시'] = data['계획출발일시']

In [5]:
temp.head()

,계획도착일시,계획출발일시
0,20191101050000,20191101050000
1,20191101050200,20191101050230
2,20191101050400,20191101050430
3,20191101050630,20191101050700
4,20191101050930,20191101051000


In [6]:
temp['계획출발일시'] = data.groupby(['운행일자', '열차번호'])['계획출발일시'].shift(1)
temp.head()

,계획도착일시,계획출발일시
0,20191101050000,NaN
1,20191101050200,2.019110e+13
2,20191101050400,2.019110e+13
3,20191101050630,2.019110e+13
4,20191101050930,2.019110e+13


In [7]:
temp[temp['계획출발일시'].isnull()]

,계획도착일시,계획출발일시
0,20191101050000,NaN
21,20191101054600,NaN
84,20191101053100,NaN
98,20191101052800,NaN
124,20191101053000,NaN
...,...,...
974639,20191130053900,NaN
974666,20191130115800,NaN
974721,20191130062700,NaN
974736,20191130113800,NaN


In [8]:
data.iloc[20]

운행일자            20191101
요일                     금
열차번호                  K1
역                     인천
출도착구분                 종착
계획도착일시    20191101054630
실제도착일시    20191101054742
계획출발일시    20191101054630
실제출발일시                  
Name: 20, dtype: object

In [9]:
data.iloc[83]

운행일자            20191101
요일                     금
열차번호                 K10
역                    소요산
출도착구분                 종착
계획도착일시    20191101082100
실제도착일시    20191101082259
계획출발일시    20191101082100
실제출발일시                  
Name: 83, dtype: object

In [10]:
temp.fillna(0, inplace=True)

In [11]:
temp['계획출발일시'] = temp['계획출발일시'].astype('int64')

In [12]:
temp[temp['계획출발일시'].isnull()]

,계획도착일시,계획출발일시


In [13]:
def time_diff(x):
  if x[1] == 0:
    result= 0
  else:
    a = dt.datetime.strptime(str(x[0]), '%Y%m%d%H%M%S')
    b = dt.datetime.strptime(str(x[1]), '%Y%m%d%H%M%S')
    c = a - b
    result = c.seconds/60 
  return result

In [14]:
data['역간소요시간'] = temp.apply(lambda x: time_diff(x), axis = 1)

In [15]:
data['역간소요시간'].describe()

count    974846.000000
mean          2.030980
std           5.123833
min           0.000000
25%           1.500000
50%           2.000000
75%           2.500000
max        1439.500000
Name: 역간소요시간, dtype: float64

In [16]:
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,1.5
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,2.0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,2.5


값이 이상함 --> 확인해보자

In [17]:
data[data['역간소요시간'].isnull()]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간


In [18]:
data[data['역간소요시간'] > 50].shape

(12, 10)

In [19]:
data[(data['역간소요시간'] > 50) & (data['출도착구분'] == '시발')].shape

(0, 10)

In [20]:
data[(data['역간소요시간'] > 50) & (data['출도착구분'] == '종착')].shape

(0, 10)

In [21]:
# 중복 데이터 수!!!!!! 제거하자!
data[(data['역간소요시간'] > 50) & (data['출도착구분'] == '정차')].shape

(12, 10)

In [22]:
data[(data['역간소요시간'] > 50) & (data['출도착구분'] == '정차')]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간
25686,20191101,금,K688,서정리,정차,20191101175830,20191101180056,20191101175900,20191101180654,1439.5
171489,20191106,수,K1901,수원,정차,20191106063900,20191106064225,20191106064000,20191106064748,1439.0
232468,20191108,금,K115,덕계,정차,20191108115130,20191108122705,20191108115200,20191108122808,1439.5
234508,20191108,금,K127,덕정,정차,20191108124800,20191108131700,20191108124830,20191108132130,1439.5
281532,20191109,토,K618,오산,정차,20191109084600,20191109084832,20191109084630,20191109085333,1439.5
366418,20191112,화,K1901,두정,정차,20191112072530,20191112073048,20191112072600,20191112073604,1439.5
482392,20191115,금,K666,두정,정차,20191115142500,20191115143257,20191115142530,20191115143334,1439.5
511214,20191116,토,K633,오산,정차,20191116103030,20191116103159,20191116103500,20191116104208,1435.5
514535,20191116,토,K696,서정리,정차,20191116201200,20191116202205,20191116201230,20191116202828,1439.5
639999,20191120,수,K600,직산,정차,20191120050830,20191120050943,20191120050900,20191120051604,1439.5


In [23]:
# index 25686를 확인해보면, 중복된 데이터가 문제 
data.iloc[25680:25690]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간
25680,20191101,금,K688,두정,정차,20191101172630,20191101173243,20191101172700,20191101173329,4.5
25681,20191101,금,K688,직산,정차,20191101173100,20191101173742,20191101173130,20191101173840,4.0
25682,20191101,금,K688,성환,정차,20191101173530,20191101174234,20191101173600,20191101174319,4.0
25683,20191101,금,K688,평택,정차,20191101174330,20191101175049,20191101174900,20191101175150,7.5
25684,20191101,금,K688,지제,정차,20191101175400,20191101175527,20191101175430,20191101175547,5.0
25685,20191101,금,K688,서정리,정차,20191101175830,20191101180056,20191101175900,20191101180654,4.0
25686,20191101,금,K688,서정리,정차,20191101175830,20191101180056,20191101175900,20191101180654,1439.5
25687,20191101,금,K688,송탄,정차,20191101180130,20191101180923,20191101180200,20191101180954,2.5
25688,20191101,금,K688,진위,정차,20191101180530,20191101181305,20191101180600,20191101181356,3.5
25689,20191101,금,K688,오산,정차,20191101180900,20191101181726,20191101180930,20191101181759,3.0


In [24]:
data.drop_duplicates(['운행일자', '열차번호', '역', '출도착구분', '계획도착일시', '계획출발일시'], inplace = True)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 974834 entries, 0 to 974845
Data columns (total 10 columns):
운행일자      974834 non-null int64
요일        974834 non-null object
열차번호      974834 non-null object
역         974834 non-null object
출도착구분     974834 non-null object
계획도착일시    974834 non-null int64
실제도착일시    974834 non-null object
계획출발일시    974834 non-null int64
실제출발일시    974834 non-null object
역간소요시간    974834 non-null float64
dtypes: float64(1), int64(3), object(6)
memory usage: 81.8+ MB


In [26]:
data['역간소요시간'].describe()

count    974834.000000
mean          2.013290
std           0.912389
min           0.000000
25%           1.500000
50%           2.000000
75%           2.500000
max           8.000000
Name: 역간소요시간, dtype: float64

# 누적시간 구하기

In [27]:
def cum_time(data):
    result = []
    value = 0

    for row in data:
        if row == 0:
            value = 0
            result.append(value)
        else:
            value += row
            result.append(value)
    return result
        

data['누적소요시간'] = cum_time(data['역간소요시간'])

In [28]:
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,1.5,3.5
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,2.0,5.5
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,2.5,8.0


# 급행여부 컬럼 추가하기

In [29]:
print(len(data['열차번호'].unique()))
data['열차번호'].value_counts()

1304


K241     1890
K84      1890
K16      1881
K229     1881
K10      1881
         ... 
K826       36
K655       36
K833       36
K908       34
K671       26
Name: 열차번호, Length: 1304, dtype: int64

In [30]:
# 열차번호마다 통과가 포함된 역의 수
data.groupby(['운행일자', '열차번호'])['출도착구분'].value_counts().head()

운행일자      열차번호  출도착구분
20191101  K1    정차       19
                시발        1
                종착        1
          K10   정차       60
                시발        1
Name: 출도착구분, dtype: int64

K10의 통과 1은 마전역에 해당된다. 이는 애당초 중간분기점을 승격시킨, 신호소로만 기능하는 역이다 보니 승강장이 없으며, 그러므로 수도권 전철 1호선 전동차를 비롯한 모든 여객열차는 이 역에 정차하지 않는다.

In [31]:
data.loc[(data['운행일자'] == 20191101) & (data['열차번호'] == 'K10')].tail(30)

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간
54,20191101,금,K10,1동대,정차,20191101070400,20191101070615,20191101070430,20191101070648,1.5,61.5
55,20191101,금,K10,동묘앞,정차,20191101070530,20191101070754,20191101070600,20191101070820,1.0,62.5
56,20191101,금,K10,신설동,정차,20191101070730,20191101070933,20191101070800,20191101070957,1.5,64.0
57,20191101,금,K10,제기동,정차,20191101070930,20191101071118,20191101071000,20191101071154,1.5,65.5
58,20191101,금,K10,1지청량리,정차,20191101071130,20191101071327,20191101071200,20191101071347,1.5,67.0
59,20191101,금,K10,회기,정차,20191101071430,20191101071559,20191101071500,20191101071631,2.5,69.5
60,20191101,금,K10,외대앞,정차,20191101071630,20191101071751,20191101071700,20191101071824,1.5,71.0
61,20191101,금,K10,신이문,정차,20191101071800,20191101071931,20191101071830,20191101072015,1.0,72.0
62,20191101,금,K10,석계,정차,20191101072030,20191101072202,20191101072100,20191101072307,2.0,74.0
63,20191101,금,K10,광운대,정차,20191101072300,20191101072443,20191101072400,20191101072531,2.0,76.0


하루에 동일한 열차번호를 가진 열차가 운행되지는 않는 것 같다!

In [32]:
tab = data[data['출도착구분'] == '통과'].groupby(['운행일자', '열차번호'])['출도착구분'].count()
tab = pd.DataFrame(tab)

data['급행여부'] = 0
for date, value in tab[tab['출도착구분'] != 1].index.values:
    idx = data.loc[(data['운행일자'] == date) & (data['열차번호'] == value)].index
    data.loc[idx, '급행여부'] = 1

In [33]:
tab

출도착구분
운행일자     열차번호        
20191101 K10        1
         K1002      4
         K1004     10
         K1005     10
         K1006     10
...               ...
20191130 K916       1
         K95        1
         K96        1
         K971       1
         K979       1

[10717 rows x 1 columns]

In [34]:
# K1004는 급행열차로 급행인지 확인
data.loc[(data['운행일자'] == 20191101) & (data['열차번호'] == 'K1004')]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
98,20191101,금,K1004,동인천,시발,20191101052800,,20191101052800,20191101052828,0.0,0.0,1
99,20191101,금,K1004,도원,통과,20191101052930,20191101052950,20191101052930,20191101052950,1.5,1.5,1
100,20191101,금,K1004,제물포,정차,20191101053100,20191101053124,20191101053130,20191101053205,1.5,3.0,1
101,20191101,금,K1004,도화,통과,20191101053300,20191101053309,20191101053300,20191101053309,1.5,4.5,1
102,20191101,금,K1004,주안,정차,20191101053400,20191101053418,20191101053430,20191101053505,1.0,5.5,1
103,20191101,금,K1004,간석,통과,20191101053600,20191101053614,20191101053600,20191101053614,1.5,7.0,1
104,20191101,금,K1004,동암,정차,20191101053730,20191101053744,20191101053800,20191101053827,1.5,8.5,1
105,20191101,금,K1004,백운,통과,20191101054000,20191101054000,20191101054000,20191101054000,2.0,10.5,1
106,20191101,금,K1004,부평,정차,20191101054130,20191101054213,20191101054200,20191101054303,1.5,12.0,1
107,20191101,금,K1004,부개,통과,20191101054330,20191101054435,20191101054330,20191101054435,1.5,13.5,1


In [35]:
# K10은 일반열차로 급행이 아닌지 확인
data.loc[(data['운행일자'] == 20191101) & (data['열차번호'] == 'K10')]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
21,20191101,금,K10,인천,시발,20191101054600,,20191101054600,20191101054623,0.0,0.0,0
22,20191101,금,K10,동인천,정차,20191101054930,20191101055019,20191101055000,20191101055109,3.5,3.5,0
23,20191101,금,K10,도원,정차,20191101055130,20191101055236,20191101055200,20191101055300,1.5,5.0,0
24,20191101,금,K10,제물포,정차,20191101055330,20191101055458,20191101055400,20191101055526,1.5,6.5,0
25,20191101,금,K10,도화,정차,20191101055530,20191101055653,20191101055600,20191101055733,1.5,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
79,20191101,금,K10,지행,정차,20191101081030,20191101081139,20191101081100,20191101081214,4.5,110.5,0
80,20191101,금,K10,동두천중앙,정차,20191101081230,20191101081404,20191101081300,20191101081442,1.5,112.0,0
81,20191101,금,K10,보산,정차,20191101081500,20191101081607,20191101081530,20191101081639,2.0,114.0,0
82,20191101,금,K10,동두천,정차,20191101081730,20191101081834,20191101081800,20191101081904,2.0,116.0,0


# 역이름 제거하기

In [36]:
data['역'].unique()

array(['구로', '구일', '개봉', '오류동', '온수', '역곡', '소사', '부천', '중동', '송내', '부개',
       '부평', '백운', '동암', '간석', '주안', '도화', '제물포', '도원', '동인천', '인천',
       '신도림', '영등포', '신길', '대방', '노량진', '용산', '남영', '지하서울', '시청', '종각',
       '1종로3가', '종로5가', '1동대', '동묘앞', '신설동', '제기동', '1지청량리', '회기', '외대앞',
       '신이문', '석계', '광운대', '월계', '녹천', '창동', '방학', '도봉', '도봉산', '망월사',
       '회룡', '의정부', '가능', '녹양', '양주', '마전', '덕계', '덕정', '지행', '동두천중앙',
       '보산', '동두천', '소요산', '가산디지털', '독산', '금천구청', '석수', '관악', '안양', '군포',
       '의왕', '수원', '세류', '병점', '세마', '오산대', '오산', '진위', '송탄', '서정리', '지제',
       '평택', '성환', '직산', '두정', '천안', '봉명', '쌍용나사렛', '아산', '배방', '온양온천',
       '신창', '문산', '파주', '월롱', '금촌', '금릉', '운정', '야당', '탄현', '일산', '풍산',
       '백마', '곡산', '대곡', '능곡', '행신', '강매', '화전', '수색', '디지털시티', '가좌',
       '신촌', '서울', '화서', '성균관대', '당정', '금정', '명학', '서동탄', '홍대입구', '서강대',
       '공덕', '효창공원앞', '이촌', '서빙고', '한남', '옥수', '응봉', '왕십리', '청량리', '중랑',
       '상봉', '망우', '1양원', '구리', '도농', '1양정', '덕소', '도심', '팔

이름수정 필요: 지하서울, 1종로3가, 1동대, 1지청량리

In [37]:
data[data['역'] == '청량리']

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
618782,20191119,화,U9531,청량리,통과,20191119114002,20191119113336,20191119114002,20191119113336,3.033333,94.033333,1


In [38]:
data[data['역'] == '1지청량리']

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
58,20191101,금,K10,1지청량리,정차,20191101071130,20191101071327,20191101071200,20191101071347,1.5,67.0,0
725,20191101,금,K103,1지청량리,정차,20191101113830,20191101114022,20191101113900,20191101114057,2.5,52.5,0
996,20191101,금,K104,1지청량리,정차,20191101131430,20191101131721,20191101131500,20191101131755,1.5,67.0,0
1218,20191101,금,K105,1지청량리,정차,20191101114830,20191101114951,20191101114900,20191101115028,2.5,26.5,0
1919,20191101,금,K108,1지청량리,정차,20191101133400,20191101133735,20191101133430,20191101133810,1.5,67.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
974648,20191130,토,S914,1지청량리,정차,20191130055700,20191130055911,20191130055730,20191130055924,1.5,14.0,0
974703,20191130,토,S92,1지청량리,정차,20191130132330,20191130133657,20191130132400,20191130133809,1.5,67.0,0
974730,20191130,토,S922,1지청량리,정차,20191130064500,20191130064459,20191130064530,20191130064523,1.5,14.0,0
974753,20191130,토,S97,1지청량리,정차,20191130121830,20191130122059,20191130121900,20191130122139,2.5,32.0,0


서울이랑 지하서울은 동일하므로 컬럼명 '서울'로 변경하기

In [39]:
data[data['역'] == '서울']

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
13072,20191101,금,K2201,서울,종착,20191101061030,20191101061300,20191101061030,,5.5,51.000000,0
13073,20191101,금,K2202,서울,시발,20191101054500,,20191101054500,20191101054523,0.0,0.000000,0
13116,20191101,금,K2203,서울,종착,20191101064800,20191101065112,20191101064800,,5.0,50.500000,0
13117,20191101,금,K2204,서울,시발,20191101063200,,20191101063200,20191101063233,0.0,0.000000,0
13160,20191101,금,K2205,서울,종착,20191101071400,20191101071709,20191101071400,,5.5,50.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
956595,20191130,토,K2232,서울,시발,20191130210100,,20191130210100,20191130210140,0.0,0.000000,0
956638,20191130,토,K2233,서울,종착,20191130232130,20191130232217,20191130232130,,5.5,53.666667,0
956639,20191130,토,K2234,서울,시발,20191130221500,,20191130221500,20191130221553,0.0,0.000000,0
956682,20191130,토,K2235,서울,종착,20191201003100,20191201003014,20191201003100,,5.5,54.000000,0


In [40]:
data[data['역'] == '지하서울']

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
49,20191101,금,K10,지하서울,정차,20191101065300,20191101065410,20191101065330,20191101065432,2.5,53.0,0
734,20191101,금,K103,지하서울,정차,20191101115700,20191101115913,20191101115730,20191101115943,2.0,66.5,0
987,20191101,금,K104,지하서울,정차,20191101125600,20191101125831,20191101125630,20191101125913,2.5,53.0,0
1227,20191101,금,K105,지하서울,정차,20191101120700,20191101121107,20191101120730,20191101121133,2.0,40.5,0
1910,20191101,금,K108,지하서울,정차,20191101131530,20191101131816,20191101131600,20191101131859,2.5,53.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
974639,20191130,토,S914,지하서울,시발,20191130053900,,20191130053900,20191130054215,0.0,0.0,0
974694,20191130,토,S92,지하서울,정차,20191130130500,20191130131441,20191130130530,20191130131546,2.5,53.0,0
974721,20191130,토,S922,지하서울,시발,20191130062700,,20191130062700,20191130062801,0.0,0.0,0
974762,20191130,토,S97,지하서울,정차,20191130123700,20191130124155,20191130123730,20191130124232,2.0,46.0,0


- 지울 역

정차하지 않는 역: 마전
통과하는 역: 

['청량리', '양수', '신원', '국수', '아신', '오빈', '양평', '원덕', '용문', '문산', '파주', '월롱', '금촌', '금릉', '운정', '야당', '탄현', '일산', '풍산', '백마', '곡산', '대곡', '능곡', '행신', '강매', '화전', '수색', '가좌','신촌',  '홍대입구', '서강대', '공덕', '효창공원앞', '이촌', '서빙고', '한남', '옥수', '응봉', '왕십리'','중랑', '상봉', '망우', '1양원', '구리', '도농', '1양정', '덕소', '도심', '팔당', '운길산', '디지털시티']

In [41]:
tab = pd.DataFrame(data['역'].value_counts())

In [42]:
tab.shape[0]

150

In [43]:
# 역이름이 하나만 있는 경우
station = tab[tab['역'] == 1].index.values
station

array(['망우', '상봉', '서강대', '중랑', '도심', '응봉', '원덕', '팔당', '홍대입구', '양평',
       '도농', '양수', '용문', '1양원', '서빙고', '옥수', '이촌', '덕소', '오빈', '효창공원앞',
       '구리', '아신', '청량리', '국수', '신원', '한남', '1양정', '운길산', '왕십리', '공덕'],
      dtype=object)

In [44]:
len(station)

30

In [45]:
data.shape[0] - data['역'].apply(lambda x: x not in station).sum()

30

In [46]:
data = data[data['역'].apply(lambda x: not x in station)]

In [47]:
data.shape[0]

974804

추가로 지울 역: 

['문산', '파주', '월롱', '금촌', '금릉', '운정', '야당', '탄현', '일산', '풍산',
 '백마', '곡산', '대곡', '능곡', '행신', '강매', '화전', '수색', '가좌','신촌', '도농']

In [48]:
remove_station = ['문산', '파주', '월롱', '금촌', '금릉', '운정', '야당', '탄현', '일산', '풍산', '백마', '곡산', 
                  '대곡', '능곡', '행신', '강매', '화전', '수색', '가좌','신촌', '도농', '마전', '디지털시티']

In [49]:
data['역'].isin(remove_station).value_counts()

False    941283
True      33521
Name: 역, dtype: int64

In [50]:
data['역'].apply(lambda x: x not in remove_station).sum()

941283

In [51]:
data = data[data['역'].apply(lambda x: not x in remove_station)]

In [52]:
data.shape[0]

941283

In [53]:
len(data['역'].unique())

98

# 역이름 수정하기

In [54]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0,0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,1.5,3.5,0
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,2.0,5.5,0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,2.5,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
974841,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,1.5,90.5,0
974842,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,2.5,93.0,0
974843,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,2.0,95.0,0
974844,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,1.5,96.5,0


In [55]:
data['역'] = data['역'].apply(lambda x: x.replace('1',''))
data['역'] = data['역'].apply(lambda x: x.replace('지청량리','청량리'))
data['역'] = data['역'].apply(lambda x: x.replace('지하서울','서울'))
data['역'] = data['역'].apply(lambda x: x.replace('동대','동대문'))

In [56]:
data['역'].unique()

array(['구로', '구일', '개봉', '오류동', '온수', '역곡', '소사', '부천', '중동', '송내', '부개',
       '부평', '백운', '동암', '간석', '주안', '도화', '제물포', '도원', '동인천', '인천',
       '신도림', '영등포', '신길', '대방', '노량진', '용산', '남영', '서울', '시청', '종각',
       '종로3가', '종로5가', '동대문', '동묘앞', '신설동', '제기동', '청량리', '회기', '외대앞',
       '신이문', '석계', '광운대', '월계', '녹천', '창동', '방학', '도봉', '도봉산', '망월사',
       '회룡', '의정부', '가능', '녹양', '양주', '덕계', '덕정', '지행', '동두천중앙', '보산',
       '동두천', '소요산', '가산디지털', '독산', '금천구청', '석수', '관악', '안양', '군포', '의왕',
       '수원', '세류', '병점', '세마', '오산대', '오산', '진위', '송탄', '서정리', '지제', '평택',
       '성환', '직산', '두정', '천안', '봉명', '쌍용나사렛', '아산', '배방', '온양온천', '신창',
       '화서', '성균관대', '당정', '금정', '명학', '서동탄'], dtype=object)

# 환승가능 컬럼

In [58]:
transfer = pd.read_csv("./raw data/환승.csv")

In [59]:
transfer['역'].unique()

array(['소요산', '동두천', '보산', '동두천중앙', '지행', '덕정', '덕계', '양주', '녹양', '가능',
       '의정부', '회룡', '망월사', '도봉산', '도봉', '방학', '창동', '녹천', '월계', '광운대',
       '석계', '신이문', '외대앞', '회기', '청량리', '제기동', '신설동', '동묘앞', '동대문',
       '종로5가', '종로3가', '종각', '시청', '서울', '남영', '용산', '노량진', '대방', '신길',
       '영등포', '신도림', '구로', '구일', '개봉', '오류동', '온수', '역곡', '소사', '부천',
       '중동', '송내', '부개', '부평', '백운', '동암', '간석', '주안', '도화', '제물포', '도원',
       '동인천', '인천', '가산디지털', '독산', '금천구청', '광명', '석수', '관악', '안양', '명학',
       '금정', '군포', '당정', '의왕', '성균관대', '화서', '수원', '세류', '병점', '서동탄',
       '세마', '오산대', '오산', '진위', '송탄', '서정리', '지제', '평택', '성환', '직산', '두정',
       '천안', '봉명', '쌍용나사렛', '아산', '배방', '온양온천', '신창', '서울역'], dtype=object)

In [60]:
data['역'].unique()

array(['구로', '구일', '개봉', '오류동', '온수', '역곡', '소사', '부천', '중동', '송내', '부개',
       '부평', '백운', '동암', '간석', '주안', '도화', '제물포', '도원', '동인천', '인천',
       '신도림', '영등포', '신길', '대방', '노량진', '용산', '남영', '서울', '시청', '종각',
       '종로3가', '종로5가', '동대문', '동묘앞', '신설동', '제기동', '청량리', '회기', '외대앞',
       '신이문', '석계', '광운대', '월계', '녹천', '창동', '방학', '도봉', '도봉산', '망월사',
       '회룡', '의정부', '가능', '녹양', '양주', '덕계', '덕정', '지행', '동두천중앙', '보산',
       '동두천', '소요산', '가산디지털', '독산', '금천구청', '석수', '관악', '안양', '군포', '의왕',
       '수원', '세류', '병점', '세마', '오산대', '오산', '진위', '송탄', '서정리', '지제', '평택',
       '성환', '직산', '두정', '천안', '봉명', '쌍용나사렛', '아산', '배방', '온양온천', '신창',
       '화서', '성균관대', '당정', '금정', '명학', '서동탄'], dtype=object)

In [61]:
print(transfer.shape[0])
transfer['역'].isin(data['역']).sum()

160


158

In [62]:
transfer.head()

,역,환승가능
0,소요산,0
1,동두천,0
2,보산,0
3,동두천중앙,0
4,지행,0


In [63]:
transfer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 2 columns):
역       160 non-null object
환승가능    160 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.6+ KB


In [64]:
transfer['환승가능'].value_counts()

0    109
2     37
3     10
4      4
Name: 환승가능, dtype: int64

In [65]:
# 병합하기

data = data.merge(transfer, on='역',how='left')
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2
2,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0,0,0
3,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0,0,0
4,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,1.5,3.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666267,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,2.0,95.0,0,0
1666268,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,1.5,96.5,0,0
1666269,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,1.5,96.5,0,0
1666270,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1666272 entries, 0 to 1666271
Data columns (total 13 columns):
운행일자      1666272 non-null int64
요일        1666272 non-null object
열차번호      1666272 non-null object
역         1666272 non-null object
출도착구분     1666272 non-null object
계획도착일시    1666272 non-null int64
실제도착일시    1666272 non-null object
계획출발일시    1666272 non-null int64
실제출발일시    1666272 non-null object
역간소요시간    1666272 non-null float64
누적소요시간    1666272 non-null float64
급행여부      1666272 non-null int64
환승가능      1666272 non-null int64
dtypes: float64(2), int64(5), object(6)
memory usage: 178.0+ MB


# 출발행 ex) 인천행

In [67]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2
2,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0,0,0
3,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,2.0,2.0,0,0
4,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,1.5,3.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666267,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,2.0,95.0,0,0
1666268,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,1.5,96.5,0,0
1666269,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,1.5,96.5,0,0
1666270,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0


In [68]:
tab = data.loc[data['출도착구분'] == '시발', ['운행일자', '열차번호', '역']]
tab

,운행일자,열차번호,역
0,20191101,K1,구로
1,20191101,K1,구로
42,20191101,K10,인천
43,20191101,K10,인천
165,20191101,K1002,부천
...,...,...,...
1666025,20191130,S922,서울
1666054,20191130,S97,양주
1666055,20191130,S97,양주
1666163,20191130,S98,인천


In [69]:
tab.rename(columns = {'역': '출발행'}, inplace = True)

In [70]:
tab.columns

Index(['운행일자', '열차번호', '출발행'], dtype='object')

In [71]:
data = data.merge(tab, on = ['운행일자', '열차번호'], how = 'left')

In [72]:
# nan값이 없는지 확인
data[data['출발행'].isnull()]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행
49934,20191101,금,K2201,서울,종착,20191101061030,20191101061300,20191101061030,,5.5,51.000000,0,4,NaN
49936,20191101,금,K2203,서울,종착,20191101064800,20191101065112,20191101064800,,5.0,50.500000,0,4,NaN
49938,20191101,금,K2205,서울,종착,20191101071400,20191101071709,20191101071400,,5.5,50.500000,0,4,NaN
49940,20191101,금,K2207,서울,종착,20191101080430,20191101080922,20191101080430,,5.5,50.500000,0,4,NaN
49942,20191101,금,K2209,서울,종착,20191101082900,20191101082733,20191101082900,,5.5,52.500000,0,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023488,20191130,토,K2227,서울,종착,20191130195230,20191130195506,20191130195230,,5.5,24.000000,0,4,NaN
3023490,20191130,토,K2229,서울,종착,20191130205230,20191130205214,20191130205230,,5.5,24.000000,0,4,NaN
3023698,20191130,토,K2231,서울,종착,20191130215230,20191130215415,20191130215230,,5.5,24.000000,0,4,NaN
3023700,20191130,토,K2233,서울,종착,20191130232130,20191130232217,20191130232130,,5.5,53.666667,0,4,NaN


In [73]:
data.loc[data['열차번호'] == 'K2201', ['역', '출도착구분']]

,역,출도착구분
49934,서울,종착
147668,서울,종착
241686,서울,종착
344784,서울,종착
451598,서울,종착
558412,서울,종착
665226,서울,종착
772040,서울,종착
869774,서울,종착
963792,서울,종착


디지털시티와 서울역의 경우 출발역에 대한 정보를 가지고 있지 않음
왜 그런가? --> 디지털시티는 1호선에 존재하지 않음 --> 제거하자!

In [74]:
data.shape

(3076380, 14)

In [75]:
data.dropna(inplace = True)

# 종착행

In [76]:
tab = data.loc[data['출도착구분'] == '종착', ['운행일자', '열차번호', '역']]
tab

,운행일자,열차번호,역
80,20191101,K1,인천
81,20191101,K1,인천
82,20191101,K1,인천
83,20191101,K1,인천
326,20191101,K10,소요산
...,...,...,...
3076161,20191130,S97,인천
3076376,20191130,S98,양주
3076377,20191130,S98,양주
3076378,20191130,S98,양주


In [77]:
tab.rename(columns = {'역': '종착행'}, inplace = True)

In [78]:
tab.columns

Index(['운행일자', '열차번호', '종착행'], dtype='object')

In [79]:
data = data.merge(tab, on = ['운행일자', '열차번호'], how = 'left')

In [80]:
data.head(30)

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
5,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
6,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
7,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
8,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
9,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천


In [81]:
data[data['종착행'].isnull()]

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행
196676,20191101,금,K2202,서울,시발,20191101054500,,20191101054500,20191101054523,0.0,0.0,0,4,서울,NaN
196677,20191101,금,K2204,서울,시발,20191101063200,,20191101063200,20191101063233,0.0,0.0,0,4,서울,NaN
196678,20191101,금,K2206,서울,시발,20191101065600,,20191101065600,20191101065642,0.0,0.0,0,4,서울,NaN
196679,20191101,금,K2208,서울,시발,20191101072400,,20191101072400,20191101072803,0.0,0.0,0,4,서울,NaN
197552,20191101,금,K2210,서울,시발,20191101073900,,20191101073900,20191101074443,0.0,0.0,0,4,서울,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10653492,20191130,토,K2228,서울,시발,20191130192000,,20191130192000,20191130192108,0.0,0.0,0,4,서울,NaN
10654317,20191130,토,K2230,서울,시발,20191130203000,,20191130203000,20191130203052,0.0,0.0,0,4,서울,NaN
10654318,20191130,토,K2232,서울,시발,20191130210100,,20191130210100,20191130210140,0.0,0.0,0,4,서울,NaN
10654319,20191130,토,K2234,서울,시발,20191130221500,,20191130221500,20191130221553,0.0,0.0,0,4,서울,NaN


In [82]:
data.loc[data['열차번호'] == 'K2202', ['역', '출도착구분']]

,역,출도착구분
196676,서울,시발
536376,서울,시발
865712,서울,시발
1231524,서울,시발
1607700,서울,시발
1983876,서울,시발
2360052,서울,시발
2736228,서울,시발
3075928,서울,시발
3405264,서울,시발


얘도 위와 같은 이유이므로, 제거해주자!

In [83]:
data.dropna(inplace = True)

In [84]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10821626,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0,인천,양주
10821627,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0,인천,양주
10821628,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0,인천,양주
10821629,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,98.5,0,0,인천,양주


# 열차 방향 컬럼 만들기

In [85]:
# 열차 방향에 대한 컬럼 만들기
temp = data.groupby(['출발행', '종착행'])['운행일자'].count()
temp[50:]

출발행  종착행
소요산  광운대      31040
     구로       35192
     인천     1015488
신창   광운대       9480
     구로        5832
     병점        2394
     용산        9546
     천안         483
     청량리      93150
양주   광운대      17472
     구로       54096
     인천      741200
영등포  인천       22080
용산   구로       25648
     동인천     990912
     병점        4200
     부평       38880
     신창       12184
     천안       44992
의정부  광운대       4800
     인천      665792
인천   광운대      63920
     구로       88032
     동두천     457864
     동묘앞      87768
     소요산     934800
     양주      728120
     의정부     573504
     창동      186368
     청량리      25200
창동   구로        3672
     동두천      23040
     소요산       5712
     양주        8160
     인천      109200
천안   광운대       8760
     구로        1800
     병점        1209
     신창         210
     용산       19462
     청량리     101430
청량리  구로       31080
     병점       30600
     서동탄      24960
     서울        4788
     신창      168084
     천안      146664
Name: 운행일자, dtype: int64

## 하행으로 분류
- 광운대 -> 구로
- 광운대 -> 병점
- 광운대 -> 서동탄
- 광운대 -> 신창
- 광운대 -> 인천
- 광운대 -> 천안
- 구로 -> 동인천
- 구로 -> 부평
- 구로 -> 서동탄
- 구로 -> 소요산
- 구로 -> 신창
- 구로 -> 인천
- 구로 -> 천안
- 동두천 -> 광운대
- 동두천 -> 구로
- 동두천 -> 인천
- 동묘앞 -> 서동탄
- 동묘앞 -> 인천
- 병점 -> 신창
- 병점 -> 천안
- 서울 -> 서동탄
- 소요산 -> 광운대
- 소요산 -> 구로
- 소요산 -> 인천
- 신창 -> 천안
- 양주 -> 광운대
- 양주 -> 구로
- 양주 -> 인천
- 영등포 -> 인천
- 용산 -> 구로
- 용산 -> 동인천
- 용산 -> 병점
- 용산 -> 부평
- 청량리 -> 구로
- 청량리 -> 병점
- 청량리 -> 서동탄
- 청량리 -> 신창
- 청량리 -> 천안
- 청량리 -> 서울
- 용산 -> 신창
- 용산 -> 천안
- 의정부 -> 광운대
- 의정부 -> 인천
- 창동 -> 구로
- 창동 -> 인천
- 천안 -> 신창

## 상행으로 분류
- 광운대 -> 동두천
- 광운대 -> 소요산
- 광운대 -> 의정부
- 광운대 -> 창동
- 구로 -> 광운대
- 구로 -> 동두천
- 구로 -> 양주
- 구로 -> 용산
- 구로 -> 의정부
- 구로 -> 청량리
- 동인천 -> 구로
- 동인천 -> 부천
- 동인천 -> 용산
- 병점 -> 광운대
- 병점 -> 용산
- 병점 -> 청량리
- 부천 -> 용산
- 부평 -> 동두천
- 부평 -> 용산
- 서동탄 -> 광운대
- 서동탄 -> 구로
- 서동탄 -> 동묘앞
- 서동탄 -> 서울
- 서동탄 -> 청량리
- 서울 -> 광운대
- 서울 -> 동묘앞
- 서울 -> 양주
- 서울 -> 의정부
- 신창 -> 광운대
- 신창 -> 구로
- 신창 -> 병점
- 신창 -> 용산
- 신창 -> 청량리
- 인천 -> 광운대
- 인천 -> 구로
- 인천 -> 동두천
- 인천 -> 동묘앞
- 인천 -> 소요산
- 인천 -> 양주
- 인천 -> 의정부
- 인천 -> 창동
- 인천 -> 청량리
- 창동 -> 동두천
- 창동 -> 소요산
- 창동 -> 양주
- 천안 -> 광운대
- 천안 -> 구로
- 천안 -> 병점
- 천안 -> 용산
- 천안 -> 청량리

- 광운대(출발행): ['동두천', '소요산', '의정부', '창동'](상행)을 제외하고는 하행
- 천안(출발행): ['신창'](하행)을 제외하고는 상행
- 서울(출발행): ['서동탄'](하행)을 제외하고는 상행
- 창동(출발행): ['구로', '인천'](하행)을 제외하고는 상행
- 병점(출발행): ['신창', '천안'](하행)을 제외하고는 상행
- 구로(출발행): ['동인천', '부평', '서동탄', '소요산', '신창', '인천', '천안'](하행)을 제외하고는 상행



- ['동인천', '부평', '부천', '인천', '신창', '서동탄'](출발행): 모두 상행
- ['동두천', '동묘앞', '청량리', '용산', '의정부', '양주', '소요산', '영등포'](출발행): 모두 하행

In [86]:
def direction(x):
    if x[0] == '광운대':
        if x[1] in ['동두천', '소요산', '의정부', '창동']:
            result = '상행'
        else:
            result = '하행'
    elif x[0] == '천안':
        if x[1] in ['신창']:
            result = '하행'
        else:
            result = '상행'
    elif x[0] == '서울':
        if x[1] in ['서동탄']:
            result = '하행'
        else:
            result = '상행'
    elif x[0] == '창동':
        if x[1] in ['구로', '인천']:
            result = '하행'
        else:
            result = '상행'
    elif x[0] == '병점':
        if x[1] in ['신창', '천안']:
            result = '하행'
        else:
            result = '상행'
    elif x[0] == '구로':
        if x[1] in ['동인천', '부평', '서동탄', '소요산', '신창', '인천', '천안']:
            result = '하행'
        else:
            result = '상행'
    elif x[0] in ['동인천', '부평', '부천', '인천', '신창', '서동탄']:
        result = '상행'
    elif x[0] in ['동두천', '동묘앞', '청량리', '용산', '의정부', '양주', '소요산', '영등포']:
        result = '하행'
    
    return result

In [87]:
data['방향'] = data[['출발행', '종착행']].apply(lambda x: direction(x), axis = 1)

In [88]:
data['방향'].value_counts()

하행    5762976
상행    5057884
Name: 방향, dtype: int64

In [89]:
# data['방향'] = np.where(data['출발행'].isin(['광운대', '구로', '동묘앞', '서울', '구로', '용산', '청량리']), '하행', 
#                       np.where((data['출발행'] == '병점') & (data['종착행'] == '천안'), '하행', '상행'))
# data['방향'].value_counts()

# 지연여부 칼럼으로 만들기

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10820860 entries, 0 to 10821630
Data columns (total 16 columns):
운행일자      int64
요일        object
열차번호      object
역         object
출도착구분     object
계획도착일시    int64
실제도착일시    object
계획출발일시    int64
실제출발일시    object
역간소요시간    float64
누적소요시간    float64
급행여부      int64
환승가능      int64
출발행       object
종착행       object
방향        object
dtypes: float64(2), int64(5), object(9)
memory usage: 1.4+ GB


In [91]:
sorted(data['계획도착일시'].unique())

[20191101050000,
 20191101050130,
 20191101050200,
 20191101050230,
 20191101050300,
 20191101050330,
 20191101050400,
 20191101050430,
 20191101050530,
 20191101050600,
 20191101050630,
 20191101050700,
 20191101050730,
 20191101050800,
 20191101050830,
 20191101050900,
 20191101050930,
 20191101051000,
 20191101051030,
 20191101051100,
 20191101051130,
 20191101051200,
 20191101051230,
 20191101051300,
 20191101051330,
 20191101051400,
 20191101051430,
 20191101051500,
 20191101051530,
 20191101051600,
 20191101051630,
 20191101051700,
 20191101051730,
 20191101051800,
 20191101051830,
 20191101051900,
 20191101051930,
 20191101052000,
 20191101052030,
 20191101052100,
 20191101052130,
 20191101052200,
 20191101052230,
 20191101052300,
 20191101052330,
 20191101052400,
 20191101052430,
 20191101052500,
 20191101052530,
 20191101052600,
 20191101052630,
 20191101052700,
 20191101052730,
 20191101052800,
 20191101052830,
 20191101052900,
 20191101052930,
 20191101053000,
 2019110105303

In [92]:
data['실제도착일시'].unique()

array(['              ', 20191101050207, 20191101050409, ...,
       20191130142804, 20191130143059, 20191130143301], dtype=object)

In [93]:
data['실제도착일시'].apply(lambda x: str(x).strip())

0                         
1                         
2                         
3                         
4                         
                 ...      
10821626    20191130143808
10821627    20191130143808
10821628    20191130143808
10821629    20191130143808
10821630    20191130143808
Name: 실제도착일시, Length: 10820860, dtype: object

지연의 정의를 어떻게 할것인가? -> 늦게 도착했을 경우를 지연으로 하자

In [94]:
def delay(x):
    if str(x[1]).strip() != '':
        a = dt.datetime.strptime(str(x[0]), '%Y%m%d%H%M%S') 
        b = dt.datetime.strptime(str(x[1]), '%Y%m%d%H%M%S') 

        if int(x[1]) >= int(x[0]):
            c = b - a
            minutes = c.seconds/60
        else:
            c = a - b  
            minutes = -c.seconds/60
    else:
        minutes = 0
    return minutes

In [95]:
# 도착지연(분)
temp = data[['계획도착일시','실제도착일시']].apply(lambda x: delay(x), axis = 1)

이상값 확인하기

In [96]:
temp.value_counts().sort_index()

-36.633333     8
-30.350000     8
-21.666667     2
-20.800000     2
-19.883333    16
              ..
 60.883333    16
 61.033333    16
 61.216667    16
 61.250000    16
 62.566667    16
Length: 2805, dtype: int64

In [97]:
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행,방향
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행


In [98]:
# 지연여부 컬럼 만들기
data['지연여부'] = temp.apply(lambda x: x >= 1, 1, 0)

In [99]:
data.head(30)

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행,방향,지연여부
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
5,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
6,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
7,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
8,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False
9,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False


# 계획도착일시를 시간으로 바꾸기(승하차 데이터와 병합하기 위해서)

In [100]:
data['계획도착(시)'] = data['계획도착일시'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d%H%M%S').hour)

In [101]:
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,누적소요시간,급행여부,환승가능,출발행,종착행,방향,지연여부,계획도착(시)
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False,5
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False,5
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False,5
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False,5
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,0.0,0,2,구로,인천,하행,False,5


# 승하차 데이터

In [103]:
people_total = pd.read_excel("./raw data/1호선 일별 시간대별 승하차.xlsx", encoding = 'cp949', skiprows= 1)
people_total.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,승차,하차,승차.1,하차.1,승차.2,하차.2,승차.3,...,승차.17,하차.17,승차.18,하차.18,승차.19,하차.19,승차.20,하차.20,승차.21,하차.21
0,2019년 11월 01일,경부선,서울,42,2,0,0,0,0,2,...,929,216,620,116,555,101,592,81,306,30
1,2019년 11월 01일,경부선,남영,38,68,0,0,4,0,141,...,1040,806,657,445,634,428,558,370,345,289
2,2019년 11월 01일,경부선,용산,57,108,1,1,13,0,198,...,4451,3080,4049,2133,4128,1431,2715,1024,1312,366
3,2019년 11월 01일,경부선,노량진,51,113,0,0,14,0,239,...,821,1296,770,681,951,599,1253,540,586,428
4,2019년 11월 01일,경부선,대방,8,153,0,0,22,1,234,...,878,1248,581,896,654,714,574,714,217,601


In [104]:
people_total.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', '승차', '하차', '승차.1', '하차.1',
       '승차.2', '하차.2', '승차.3', '하차.3', '승차.4', '하차.4', '승차.5', '하차.5', '승차.6',
       '하차.6', '승차.7', '하차.7', '승차.8', '하차.8', '승차.9', '하차.9', '승차.10',
       '하차.10', '승차.11', '하차.11', '승차.12', '하차.12', '승차.13', '하차.13', '승차.14',
       '하차.14', '승차.15', '하차.15', '승차.16', '하차.16', '승차.17', '하차.17', '승차.18',
       '하차.18', '승차.19', '하차.19', '승차.20', '하차.20', '승차.21', '하차.21'],
      dtype='object')

In [105]:
people_total.rename(columns = {'Unnamed: 0': '운행일자',
                        'Unnamed: 1': '전철선',
                        'Unnamed: 2': '역'}, inplace = True)

In [106]:
people_total['운행일자'] = people_total['운행일자'].apply(lambda x: re.sub('년|월|일| ', '',x))

In [107]:
people_total['전철선'].value_counts()

경부선    1170
경원선     900
경인선     600
장항선     180
Name: 전철선, dtype: int64

In [108]:
people_total['역'].value_counts()

청량리     30
석수      30
성균관대    30
용산      30
소요산     30
        ..
신도림     30
병점      30
화서      30
오산대     30
온수      30
Name: 역, Length: 95, dtype: int64

In [109]:
people_total['역'].unique()

array(['서울', '남영', '용산', '노량진', '대방', '신길', '영등포', '신도림', '구로', '가산디지털단지',
       '독산', '금천구청', '석수', '관악', '안양', '명학', '금정', '군포', '당정', '의왕',
       '성균관대', '화서', '수원', '세류', '병점', '세마', '오산대', '오산', '진위', '송탄',
       '서정리', '지제', '평택', '성환', '직산', '두정', '천안', '서동탄', '광명', '구일', '개봉',
       '오류동', '온수', '역곡', '소사', '부천', '중동', '송내', '부개', '부평', '백운', '동암',
       '간석', '주안', '도화', '제물포', '도원', '동인천', '인천', '봉명', '쌍용(나사렛대)', '아산',
       '배방', '온양온천', '신창(순천향대)', '이촌', '서빙고', '한남', '옥수', '응봉', '왕십리',
       '청량리', '회기', '외대앞', '신이문', '석계', '광운대', '월계', '녹천', '방학', '도봉',
       '도봉산', '망월사', '회룡', '의정부', '가능', '녹양', '양주', '덕계', '덕정', '지행',
       '동두천중앙', '보산', '동두천', '소요산'], dtype=object)

In [110]:
# 필요없는 정보 지우기
people_total.drop(['전철선'], axis = 1, inplace = True)

In [111]:
# 불러오지 못한 시간 컬럼명 만들기
time =  [str(i) + '~' + str(i+1) for i in range(4, 24)]
time.insert(0, '1~2')
time.insert(0, '0~1')
time

['0~1',
 '1~2',
 '4~5',
 '5~6',
 '6~7',
 '7~8',
 '8~9',
 '9~10',
 '10~11',
 '11~12',
 '12~13',
 '13~14',
 '14~15',
 '15~16',
 '16~17',
 '17~18',
 '18~19',
 '19~20',
 '20~21',
 '21~22',
 '22~23',
 '23~24']

wide -> long

In [112]:
# 운행일자, 역 데이터 늘리기
value = [] 
for i in time:
    for j in range(people_total.shape[0]):
        value.append(i)

In [113]:
# 승하차 데이터 늘리기
value2, value3 = [], [] 
for i in range(2, people_total.shape[1], 2):
    value2.extend(people_total.iloc[:, i]) # 승차
    value3.extend(people_total.iloc[:, (i+1)]) # 하차

In [114]:
# long 형태로 구성하기 
people = pd.DataFrame()
for i in range(len(time)):
    people = people.append(people_total.iloc[:, :2])

In [115]:
print(len(value), len(value2), len(value3))

62700 62700 62700


In [116]:
people['시간'] = value
people['승차'] = value2
people['하차'] = value3

people.head()

,운행일자,역,시간,승차,하차
0,20191101,서울,0~1,42,2
1,20191101,남영,0~1,38,68
2,20191101,용산,0~1,57,108
3,20191101,노량진,0~1,51,113
4,20191101,대방,0~1,8,153


In [117]:
# 병합을 위한 시간컬럼 만들기
people['시간1'] = people['시간'].apply(lambda x: int(x.split('~')[0]))

In [118]:
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62700 entries, 0 to 2849
Data columns (total 6 columns):
운행일자    62700 non-null object
역       62700 non-null object
시간      62700 non-null object
승차      62700 non-null int64
하차      62700 non-null int64
시간1     62700 non-null int64
dtypes: int64(3), object(3)
memory usage: 3.3+ MB


In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10820860 entries, 0 to 10821630
Data columns (total 18 columns):
운행일자       int64
요일         object
열차번호       object
역          object
출도착구분      object
계획도착일시     int64
실제도착일시     object
계획출발일시     int64
실제출발일시     object
역간소요시간     float64
누적소요시간     float64
급행여부       int64
환승가능       int64
출발행        object
종착행        object
방향         object
지연여부       bool
계획도착(시)    int64
dtypes: bool(1), float64(2), int64(6), object(9)
memory usage: 1.5+ GB


In [120]:
people['운행일자'] = people['운행일자'].astype('int64')

In [121]:
# data의 실제도착(시)와  people의 시간1 병합하기
data = data.merge(people, how = 'left', left_on = ['운행일자', '역', '계획도착(시)'], right_on = ['운행일자', '역', '시간1'])

In [122]:
data.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,...,환승가능,출발행,종착행,방향,지연여부,계획도착(시),시간,승차,하차,시간1
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,2,구로,인천,하행,False,5,5~6,1122.0,170.0,5.0
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,2,구로,인천,하행,False,5,5~6,1122.0,170.0,5.0
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,2,구로,인천,하행,False,5,5~6,1122.0,170.0,5.0
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,2,구로,인천,하행,False,5,5~6,1122.0,170.0,5.0
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,2,구로,인천,하행,False,5,5~6,1122.0,170.0,5.0


In [123]:
data.columns

Index(['운행일자', '요일', '열차번호', '역', '출도착구분', '계획도착일시', '실제도착일시', '계획출발일시',
       '실제출발일시', '역간소요시간', '누적소요시간', '급행여부', '환승가능', '출발행', '종착행', '방향',
       '지연여부', '계획도착(시)', '시간', '승차', '하차', '시간1'],
      dtype='object')

In [124]:
# 필요없는 열 제거
data.drop(['시간1'], axis = 1, inplace = True)

미래 데이터 생성하기

In [125]:
data_train = data[data['운행일자'] < 20191125]
data_train

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,...,급행여부,환승가능,출발행,종착행,방향,지연여부,계획도착(시),시간,승차,하차
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,0,2,구로,인천,하행,False,5,5~6,1122.0,170.0
1,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,0,2,구로,인천,하행,False,5,5~6,1122.0,170.0
2,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,0,2,구로,인천,하행,False,5,5~6,1122.0,170.0
3,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,0,2,구로,인천,하행,False,5,5~6,1122.0,170.0
4,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0.0,...,0,2,구로,인천,하행,False,5,5~6,1122.0,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618144,20191124,일,S98,양주,종착,20191124143500,20191124144015,20191124143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,396.0,363.0
8618145,20191124,일,S98,양주,종착,20191124143500,20191124144015,20191124143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,396.0,363.0
8618146,20191124,일,S98,양주,종착,20191124143500,20191124144015,20191124143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,396.0,363.0
8618147,20191124,일,S98,양주,종착,20191124143500,20191124144015,20191124143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,396.0,363.0


In [126]:
data_test = data[data['운행일자'] >= 20191125]
data_test.head()

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,...,급행여부,환승가능,출발행,종착행,방향,지연여부,계획도착(시),시간,승차,하차
8618149,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1106.0,151.0
8618150,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1106.0,151.0
8618151,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1106.0,151.0
8618152,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1106.0,151.0
8618153,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1106.0,151.0


In [127]:
temp = data_train.groupby(['요일', '열차번호', '역'])['승차', '하차'].mean()
temp

승차       하차
요일 열차번호 역                        
금  K1   간석    219.250000    43.00
        개봉    741.250000   211.00
        구로   1074.500000   173.75
        구일    144.500000    45.25
        도원     97.500000    48.25
...                  ...      ...
화  S98  중동    481.000000   355.00
        창동           NaN      NaN
        청량리   988.000000  1099.00
        회기   1579.000000  1351.00
        회룡    702.666667   548.00

[243344 rows x 2 columns]

미래 데이터 승차, 하차 넣기

In [128]:
temp2 = data_test[['운행일자', '요일', '열차번호', '역', '승차', '하차']].sort_values(by = ['운행일자', '요일', '열차번호', '역'])
temp2[(temp2['열차번호'] == 'K1') & (temp2['요일'] == '금')]

,운행일자,요일,열차번호,역,승차,하차
10115619,20191129,금,K1,간석,211.0,46.0
10115620,20191129,금,K1,간석,211.0,46.0
10115621,20191129,금,K1,간석,211.0,46.0
10115622,20191129,금,K1,간석,211.0,46.0
10115623,20191129,금,K1,간석,211.0,46.0
...,...,...,...,...,...,...
10115534,20191129,금,K1,중동,366.0,63.0
10115535,20191129,금,K1,중동,366.0,63.0
10115536,20191129,금,K1,중동,366.0,63.0
10115537,20191129,금,K1,중동,366.0,63.0


In [129]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2202711 entries, 8618373 to 10820795
Data columns (total 6 columns):
운행일자    int64
요일      object
열차번호    object
역       object
승차      float64
하차      float64
dtypes: float64(2), int64(1), object(3)
memory usage: 117.6+ MB


In [130]:
data_test.drop(['승차', '하차'], axis = 1, inplace = True)

C:\Users\seung\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [131]:
data_test = data_test.merge(temp, how = 'left', right_index = True, left_on = ['요일', '열차번호', '역'])

In [132]:
data_test

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,역간소요시간,...,급행여부,환승가능,출발행,종착행,방향,지연여부,계획도착(시),시간,승차,하차
8618149,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1103.0,197.0
8618150,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1103.0,197.0
8618151,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1103.0,197.0
8618152,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1103.0,197.0
8618153,20191125,월,K1,구로,시발,20191125050000,,20191125050000,20191125050034,0.0,...,0,2,구로,인천,하행,False,5,5~6,1103.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10820855,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,557.0,452.5
10820856,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,557.0,452.5
10820857,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,557.0,452.5
10820858,20191130,토,S98,양주,종착,20191130143500,20191130143808,20191130143500,,2.0,...,0,0,인천,양주,상행,True,14,14~15,557.0,452.5


In [133]:
data = pd.concat([data_train, data_test])

In [134]:
data.to_csv('data_ver1.csv', index = False)

# 날씨 병합하기

In [135]:
weather = pd.read_csv('./raw data/날씨.csv', encoding='cp949') 

In [136]:
weather.head()

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,98,동두천,2018-11-02 11:00,13.0,0,NaN,NaN,1.5,0.0,200.0,...,NaN,542,NaN,NaN,19.1,0,NaN,NaN,NaN,NaN
1,98,동두천,2018-11-02 12:00,15.8,0,NaN,NaN,1.4,0.0,230.0,...,NaN,634,NaN,NaN,22.5,0,NaN,NaN,NaN,NaN
2,98,동두천,2018-11-02 13:00,17.6,0,NaN,NaN,1.1,0.0,230.0,...,NaN,749,NaN,NaN,21.5,0,NaN,NaN,NaN,NaN
3,98,동두천,2018-11-02 14:00,17.4,0,NaN,NaN,1.4,0.0,140.0,...,NaN,1172,NaN,NaN,22.9,0,NaN,NaN,NaN,NaN
4,98,동두천,2018-11-02 15:00,18.6,0,NaN,NaN,0.8,0.0,200.0,...,NaN,1053,NaN,NaN,21.6,0,NaN,NaN,NaN,NaN


In [137]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3430 entries, 0 to 3429
Data columns (total 37 columns):
지점               3430 non-null int64
지점명              3430 non-null object
일시               3430 non-null object
기온(°C)           3428 non-null float64
기온 QC플래그         3430 non-null int64
강수량(mm)          126 non-null float64
강수량 QC플래그        677 non-null float64
풍속(m/s)          3429 non-null float64
풍속 QC플래그         3429 non-null float64
풍향(16방위)         3429 non-null float64
풍향 QC플래그         3429 non-null float64
습도(%)            3429 non-null float64
습도 QC플래그         3430 non-null int64
증기압(hPa)         3429 non-null float64
이슬점온도(°C)        3429 non-null float64
현지기압(hPa)        3429 non-null float64
현지기압 QC플래그       3430 non-null int64
해면기압(hPa)        3429 non-null float64
해면기압 QC플래그       3430 non-null int64
일조(hr)           1584 non-null float64
일조 QC플래그         3429 non-null float64
일사(MJ/m2)        950 non-null float64
적설(cm)           89 non-null float64
3시간신적설(cm)   

In [138]:
def date_chgFormat(data):
    data['운행일자'] = data['일시'].apply(lambda x: str(x)[:10].replace('-', ''))
    data['시간'] = data['일시'].apply(lambda x: str(x)[11:13][1] if str(x)[11:13][0] == '0' else str(x)[11:13])
    data['시간'] = data['시간'].apply(lambda x: str(x) + '~' + str(int(x)+1))
   
    data.drop(['일시', '지점'], axis = 1, inplace = True)
    
    # non na 비율이 0.9인 컬럼만 가져오기
    temp = data.count()/data.shape[0]
    data = data[temp[temp > 0.9].index.to_list()]
        
    # 데이터 타입 맞춰주기
    data['운행일자'] = data['운행일자'].astype('int64')
    
    return data

In [139]:
weather = date_chgFormat(weather)

C:\Users\seung\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [140]:
def shift_date(x):
    value = dt.datetime.strptime(str(x), '%Y%m%d')
    year = value.year + 1
    month = 11
    if value.day >= 2 and value.day <= 11:
        day = '0' + str(value.day - 1)
    elif value.day > 11 and value.day <= 30:
        day = value.day - 1
    else:
        day = 30
    
    return str(year) + str(month) + str(day)

In [141]:
weather['운행일자'] = weather['운행일자'].apply(lambda x: shift_date(x))

In [142]:
# 하루 날씨 살펴보기
weather.loc[weather['운행일자'] == '20191101', ['지점명', '기온(°C)', '시간']]

,지점명,기온(°C),시간
0,동두천,13.0,11~12
1,동두천,15.8,12~13
2,동두천,17.6,13~14
3,동두천,17.4,14~15
4,동두천,18.6,15~16
...,...,...,...
2752,천안,8.8,19~20
2753,천안,7.1,20~21
2754,천안,5.6,21~22
2755,천안,4.5,22~23


In [143]:
# 시간별 기온차이가 많이 나는가?
weather[['운행일자', '지점명', '시간', '기온(°C)']].head(30)

,운행일자,지점명,시간,기온(°C)
0,20191101,동두천,11~12,13.0
1,20191101,동두천,12~13,15.8
2,20191101,동두천,13~14,17.6
3,20191101,동두천,14~15,17.4
4,20191101,동두천,15~16,18.6
5,20191101,동두천,16~17,18.4
6,20191101,동두천,17~18,16.4
7,20191101,동두천,18~19,13.6
8,20191101,동두천,19~20,11.2
9,20191101,동두천,20~21,9.9


In [144]:
# 수집한 자료의 지역 살펴보기
weather['지점명'].value_counts()

서울     686
동두천    686
인천     686
수원     686
천안     686
Name: 지점명, dtype: int64

날씨와 최종데이터셋을 병합할 때의 기준(지도를 보고 결정함)

수원: ['수원', '세류', '화서', '성균관대', '당정', '군포', '금정', '명학', '병점', '서동탄', '세마', '오산대', '오산', '진위']

동두천: ['소요산', '동두천', '보산', '동두천중양', '지행', '덕정', '덕계', '양주', '녹양', '가능']

천안: ['신창', '온양온천', '배방', '아산', '쌍용', '봉명', '천안', '두정', '직산', '성환', '송탄', '평택', '지제', '서정리']

인천: ['인천', '동인천', '도원', '제물포', '도화', '주안', '간석', '동암역', '백운', '부평', '부개', '송내', '중동', '부천', '소사', '역곡']

서울: ['구로', '영등포', '신도림', '용산', '노량진', '대방', '신길', '서울', '종로5가', '동대문', '종로3가', '시청', '동묘앞', '종각', '남영', '제기동', '신설동', '청량리', '오류동', '구일', '온수', '광운대', '신이문', '석계', '외대앞', '회기', '안양', '금천구청', '관악', '독산', '석수', '가산디지털', '창동', '녹천', '월계', '망월사', '도봉', '회룡', '도봉산', '방학', '의정부']

In [145]:
def match_data(weather, data):   
    
#     weather = weather.merge(weather9, how = 'left', on = ['지점명', '운행일자', '시간'])

    suwon = ['수원', '세류', '화서', '성균관대', '당정', '군포', '금정', '명학', '병점', '서동탄', '세마', '오산대', '오산', '진위']
    dongdu = ['소요산', '동두천', '보산', '동두천중양', '지행', '덕정', '덕계', '양주', '녹양', '가능']
    cheonan = ['신창', '온양온천', '배방', '아산' '쌍용나사렛', '봉명', '천안', '두정', '직산', '성환', '송탄', '평택', '지제', '서정리']
    incheon = ['인천', '동인천', '도원', '제물포', '도화', '주안', '간석', '동암역', '백운', '부평', '부개', '송내', '중동', '부천', '소사', '역곡']
    seoul =  ['구로', '영등포', '신도림', '용산', '노량진', '대방', '신길', '서울', '종로5가', '동대문', '종로3가', '시청', '동묘앞',
              '종각', '남영', '제기동', '신설동', '청량리', '오류동', '구일', '온수', '광운대', '신이문', '석계', '외대앞', '회기', 
              '안양', '금천구청', '관악', '독산', '석수', '가산디지털', '창동', '녹천', '월계', '망월사', '도봉', '회룡', '도봉산',
              '방학', '의정부']
    
    weather['운행일자'] = weather['운행일자'].astype('int64')
    data['운행일자'] = data['운행일자'].astype('int64')
    
    data_suwon = data[data['역'].isin(suwon)].merge(weather[weather['지점명'] == '수원'], how = 'left', on = ['운행일자', '시간'])
    data_dongdu  = data[data['역'].isin(dongdu)].merge(weather[weather['지점명'] == '동두천'], how = 'left', on = ['운행일자', '시간'])
    data_cheonan = data[data['역'].isin(cheonan)].merge(weather[weather['지점명'] == '천안'], how = 'left', on = ['운행일자', '시간'])
    data_incheon = data[data['역'].isin(incheon)].merge(weather[weather['지점명'] == '인천'], how = 'left', on = ['운행일자', '시간'])
    data_seoul = data[data['역'].isin(seoul)].merge(weather[weather['지점명'] == '서울'], how = 'left', on = ['운행일자', '시간'])

    data = pd.concat([data_suwon, data_dongdu, data_cheonan, data_incheon, data_seoul])
    
    # NA가 발생한 행 제거
    data.dropna(inplace = True)
    
    data.reset_index(inplace = True)
    
    # 데이터 분석에 필요없는 행 제거 
    data.drop(['index', '지점명', '계획도착일시','실제도착일시', '계획출발일시', '실제출발일시', '계획도착(시)'],  axis = 1, inplace = True)
    
    return data

제거할 컬럼명: '지점', '지점명',
'기온 QC플래그', '강수량(mm)', '강수량 QC플래그', '풍속 QC플래그', '풍향 QC플래그', '습도 QC플래그', '현지기압 QC플래그', '해면기압 QC플래그', '일조(hr)', '일조 QC플래그', '일사(MJ/m2)', '적설(cm) ', '3시간신적설(cm)', '운형(운형약어)', '최저운고(100m )', '지면상태(지면상태코드)', '현상번호(국내식)', '지면온도 QC플래그', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)', '30cm 지중온도(°C)'

In [146]:
data = match_data(weather, data)
data.head()

,운행일자,요일,열차번호,역,출도착구분,역간소요시간,누적소요시간,급행여부,환승가능,출발행,...,현지기압(hPa),현지기압 QC플래그,해면기압(hPa),해면기압 QC플래그,일조 QC플래그,전운량(10분위),중하층운량(10분위),시정(10m),지면온도(°C),지면온도 QC플래그
0,20191101,금,K1907,수원,정차,7.0,41.5,1,2,용산,...,1025.3,0.0,1029.5,0.0,0.0,0.0,0.0,1195.0,11.5,0.0
1,20191101,금,K1907,수원,정차,7.0,41.5,1,2,용산,...,1025.3,0.0,1029.5,0.0,0.0,0.0,0.0,1195.0,11.5,0.0
2,20191101,금,K1907,수원,정차,7.0,41.5,1,2,용산,...,1025.3,0.0,1029.5,0.0,0.0,0.0,0.0,1195.0,11.5,0.0
3,20191101,금,K1907,수원,정차,7.0,41.5,1,2,용산,...,1025.3,0.0,1029.5,0.0,0.0,0.0,0.0,1195.0,11.5,0.0
4,20191101,금,K1907,세류,통과,2.5,44.0,1,0,용산,...,1025.3,0.0,1029.5,0.0,0.0,0.0,0.0,1195.0,11.5,0.0


In [147]:
data.to_csv('total_dataset_ver1.csv', index = False)